<a href="https://colab.research.google.com/github/torbenbillow/CBS-AML-PROJECT/blob/main/notebooks/00_welcome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.svm import SVR, SVC

from sklearn.metrics import (mean_absolute_error, mean_squared_error, r2_score,
                             accuracy_score, precision_score, recall_score,
                             f1_score, confusion_matrix, classification_report,
                             roc_auc_score, roc_curve, max_error, mean_absolute_percentage_error)

from google.colab import drive
pd.options.mode.copy_on_write = True

# LOAD DATA FROM GOOGLE DRIVE

In [32]:
print("Loading raw data from Gdrive location...")
file_id = "1Iyr7zX8u0gKWKUWCSgZUpcvbTuYBmw1V"
url = f"https://drive.google.com/uc?id={file_id}"

listings_raw = pd.read_csv(url)
print("Raw Data Shape:",listings_raw.shape)

Loading raw data from Gdrive location...
Raw Data Shape: (22684, 79)


# DATA WRANGLING

## SHOW COLUMNS AND MANUALLY REMOVE UNWANTED (DOMAIN KNOWLEDGE)

In [33]:
df = listings_raw.copy()
list(df)

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'source',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30

In [34]:
# Check size of data frame before drops
print("Before column drops:",df.shape)

Before column drops: (22684, 79)


In [35]:
# The following columns are irrelevant, and thus, we have chosen to drop them.
df = df.drop(columns=["id","listing_url","scrape_id","source","host_id","host_url","calendar_last_scraped","first_review","host_neighbourhood"])

# The following columns could impact a listing's "success" (discrimination, etc.), but due to the complexity and the scope of this project,
# we have chosen to drop it.
df = df.drop(columns=["name","picture_url","host_name","host_thumbnail_url","host_picture_url"])

# Dropping the following columns in favor of "calculated_host_listings_count," because that is the direct calculation of how many listings
# a host has at the time of scrape. The metadata also notes that this calculation is "unknown," and thereby less trustworthy.
df = df.drop(columns=["host_listings_count","host_total_listings_count"])

# Dropping in favor of "host_identitity_verified," which is a boolean that indicates whether or not the host is verified.
df = df.drop(columns=["host_verifications"])

# Dropping in favor of "neighborhood_cleansed."
df = df.drop(columns=["neighbourhood"])

# Dropping the following, as they are completely empty attributes.
df = df.drop(columns=["neighbourhood_group_cleansed","calendar_updated","license"])

# Dropping the following columns in favor of "mini_nights" and "maximum_nights."
df = df.drop(columns=["minimum_minimum_nights","maximum_minimum_nights","minimum_maximum_nights","maximum_maximum_nights","minimum_nights_avg_ntm","maximum_nights_avg_ntm"])

# Dropping the following columns, because there is no information about they represent.
df = df.drop(columns=["has_availability","availability_eoy"])

''' Dropping the following columns, because we have selected "availability_30" as our target variable. We believe this the most accurate
indicator of a listing's popularity/success, because listings will on average be booked more in the short term than in the long term.
We would be able to attribute the availability to an actual "interest level," rather than a lack of booking simply due to time considerations.
Seasonality should not matter since the scope of our listings is limited to Copenhagen, so theoretically, all listings would be impacted equally
by seasonality. '''
df = df.drop(columns=["availability_60","availability_90","availability_365"])

# Dropping the following columns in favor "number_of_reviews" for simplicity.
df = df.drop(columns=["number_of_reviews_ltm","number_of_reviews_l30d"])

# Dropping in favor "number_of_reviews" for simplicity.
df = df.drop(columns=["number_of_reviews_ly"])

# Irrelevant and would introduce data leakage.
df = df.drop(columns=["estimated_revenue_l365d"])

# Dropping the following attributes because they will not be available at the time of prediction.
df = df.drop(columns=["review_scores_rating","review_scores_accuracy","review_scores_cleanliness","review_scores_checkin","review_scores_communication","review_scores_location","review_scores_value"])

# Dropping the following columns in favor of "calculated_host_listings_count" for simplicity.
df = df.drop(columns=["calculated_host_listings_count_entire_homes","calculated_host_listings_count_private_rooms","calculated_host_listings_count_shared_rooms"])

# Dropping in favor of "number_of_reviews."
df = df.drop(columns=["reviews_per_month"])

# Check size of data frame after drops
print("After column drops:",df.shape)


After column drops: (22684, 32)


## AUDIT THE DATA TO DETERMINE WHAT NEEDS TO BE CLEANSED

In [36]:
# Create a function that will audit the raw data and provide an overview, making it easier to systematically determine what needs to be cleansed.

def audit_dataframe(df, name="DataFrame"):
    print(f"\n=== AIRBNB DATA AUDIT: {name} ===")
    print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns\n")

    # ----- SHOW NULL SUMMARY -----
    print("---- Missing Values ----")
    nulls = df.isnull().sum()
    nulls_percent = (df.isnull().mean() * 100).round(2)
    null_summary = pd.DataFrame({
        "null_count": nulls,
        "null_percent": nulls_percent,
        "dtype": df.dtypes
    })
    print(null_summary[nulls > 0].sort_values("null_percent", ascending=False))
    print("\n")

    # ----- SHOW DATA TYPES -----
    print("---- Data Types ----")
    with pd.option_context("display.max_rows", None, "display.max_columns", None):
      print(df.dtypes)
    print("\n")

    # ----- SHOW DESCRIPTIVE STATS -----
    print("---- Descriptive Statistics (Numerical) ----")
    print(df.describe(include=[np.number]).transpose())
    print("\n")

    print("---- Descriptive Statistics (Categorical) ----")
    print(df.describe(include=['object', 'category']).transpose())
    print("\n")

    # ----- SHOW UNIQUE VALUES -----
    print("---- Unique Value Counts ----")
    unique_counts = df.nunique().sort_values()
    print(unique_counts)
    print("\n")

    # ----- SHOW SAMPLE VALUES FOR EACH COLUMN -----
    print("---- Sample Values ----")
    for col in df.columns:
        print(f"{col}: {df[col].dropna().unique()[:5]} ...")
    print("\n")

    # ----- SHOW BOOLEAN DISTRIBUTION -----
    bool_cols = df.select_dtypes(include="bool").columns
    if len(bool_cols) > 0:
        print("---- Boolean Column Distributions ----")
        for col in bool_cols:
            print(f"{col}:\n{df[col].value_counts()}")
            print("")
    print("\n")

    # ----- SHOW CATEGORICAL CARDINALITY -----
    print("---- Categorical Cardinality ----")
    cat_cols = df.select_dtypes(include=["object", "category"]).columns
    for col in cat_cols:
        print(f"{col}: {df[col].nunique()} unique values")
    print("\n")

    # ----- SHOW DUPLICATES -----
    print("---- Duplicate Rows ----")
    dupes = df.duplicated().sum()
    print(f"Duplicate rows: {dupes}")
    print("\n")

    print("=== END AUDIT ===\n")


In [37]:
audit_dataframe(df, name="Airbnb Raw Data")


=== AIRBNB DATA AUDIT: Airbnb Raw Data ===
Shape: 22684 rows × 32 columns

---- Missing Values ----
                        null_count  null_percent    dtype
neighborhood_overview        14459         63.74   object
host_about                   13381         58.99   object
bathrooms                     8858         39.05  float64
beds                          8856         39.04  float64
price                         8853         39.03   object
host_response_time            7853         34.62   object
host_response_rate            7853         34.62   object
host_acceptance_rate          5288         23.31   object
host_location                 4082         18.00   object
last_review                   3146         13.87   object
bedrooms                       716          3.16  float64
description                    614          2.71   object
host_since                     529          2.33   object
host_identity_verified         529          2.33   object
host_has_profile_pic         

## FILTER THE DATASET TO REMOVE LISTINGS WITHOUT REVIEWS AND WITH NO OCCUPANCY IN THE LAST YEAR

In [39]:
'''We have chosen to remove listings without reviews in 2025 and no occupancy in the last year. We have chosen to filter "last_review" for
listings that have a review within 6 months of the scrape date and "estimated_occupancy_l365d" for listings that have a non-zero value. We have
done this to ensure we are including only listings that are active, and thereby more informative for our model.

We recognize that this approach may introduce selection bias, since the remaining dataset over-represents highly active, consistently booked listings.
As a result, our model will be trained only on relatively "successful" or "popular" listings and may systematically overestimate occupancy for
lower-activity or newly listed properties. In other words, the predictions may not generalize as well to the full population of Airbnb listings,
because the model never sees ultra low-engagement listings.'''

df = df.query(
    "last_review >= '2025-01-01' and estimated_occupancy_l365d > 0"
)

In [42]:
# Check sizes of data and filtered data
print("Row count before filtering:", len(listings_raw))
print("Row count after filtering:", len(df))

Row count before filtering: 22684
Row count after filtering: 10132


In [43]:
# We are now dropping the aforementioned columns, as they are no longer needed post-filtering.

df = df.drop(columns=["last_review","estimated_occupancy_l365d"])

## CLEAN UP "price" ATTRIBUTE



In [ ]:
# Remove $, commas, and spaces, then convert to numeric
price_clean = (
    df["price"]
    .astype(str)                             # handle existing ints / NA
    .str.replace(r'[\$,]', '', regex=True)   # remove $ and commas
    .str.strip()
)

# Convert to numeric, coercing bad values (like '<NA>') to NaN
price_numeric = pd.to_numeric(price_clean, errors="coerce")

# Store back as nullable integer
df["price"] = price_numeric.astype("Int64")

## VALIDATE PRICE ATTRIBUTE

In [ ]:
df["price"].describe()

,price
count,7872.0
mean,1374.50216
std,1167.65617
min,202.0
25%,881.0
50%,1168.0
75%,1600.0
max,63418.0


## CREATE TWO DATAFRAMES WITH DIFFERENT "price" APPROACHES

In [ ]:
# Create two new dataframes to split between approaches (dropping missing prices vs. imputing missing prices)
df_price_imputation = df.copy()
df_drop_price = df.copy()

median_price = df["price"].median()
df_price_imputation["price"] = df_price_imputation["price"].fillna(median_price).astype("Int64")
df_drop_price = df_drop_price.dropna(subset=["price"])

print("Shape of df with price imputation:", df_price_imputation.shape)
print("Shape of df with dropped blank prices:", df_drop_price.shape)

Shape of df with price imputation: (10132, 30)
Shape of df with dropped blank prices: (7872, 30)


In [ ]:
# Create a dictionary with the two split datasets.
# Instead of rewriting the pipeline for each version, we can loop over the dataset dictionary and train/evaluate all models on all variants cleanly.

datasets = {
    "drop_missing_prices": df_drop_price,
    "impute_price": df_price_imputation
}

print(type(datasets))
for name, df in datasets.items():
    print(name, type(df))

<class 'dict'>
drop_missing_prices <class 'pandas.core.frame.DataFrame'>
impute_price <class 'pandas.core.frame.DataFrame'>


## CLEAN UP BOOLEAN COLUMNS

In [ ]:
# Some cols have boolean values "f" and "t", we change to 0 or 1
bool_cols = [
    "host_is_superhost",
    "host_has_profile_pic",
    "host_identity_verified",
    "instant_bookable"
]

bool_map = {
    "t": 1,
    "f": 0,
    "true": 1,
    "false": 0,
    True: 1,
    False: 0,
}

# Fix boolean values in the datasets
for name, df in datasets.items():
  for col in bool_cols:
      df[col] = (df[col].map(bool_map).astype("Int64"))

# Fill missing superhost values with 0 in the datasets
for name, df in datasets.items():
  df["host_is_superhost"] = df["host_is_superhost"].fillna(0).astype("Int64")

# Print results
for name, df in datasets.items():
  print(f"Cleaned up column information for {name} dataframe...\n")
  for col in bool_cols:
    print(f"{col} unique values:", df[col].unique(),"\n")

Cleaned up column information for drop_missing_prices dataframe...

host_is_superhost unique values: <IntegerArray>
[1, 0]
Length: 2, dtype: Int64 

host_has_profile_pic unique values: <IntegerArray>
[1, <NA>, 0]
Length: 3, dtype: Int64 

host_identity_verified unique values: <IntegerArray>
[1, 0, <NA>]
Length: 3, dtype: Int64 

instant_bookable unique values: <IntegerArray>
[0, 1]
Length: 2, dtype: Int64 

Cleaned up column information for impute_price dataframe...

host_is_superhost unique values: <IntegerArray>
[1, 0]
Length: 2, dtype: Int64 

host_has_profile_pic unique values: <IntegerArray>
[1, <NA>, 0]
Length: 3, dtype: Int64 

host_identity_verified unique values: <IntegerArray>
[1, 0, <NA>]
Length: 3, dtype: Int64 

instant_bookable unique values: <IntegerArray>
[0, 1]
Length: 2, dtype: Int64 



In [ ]:
# We can see based on the information below, that a number of listings have a missing verification status and profile picture.
for name, df in datasets.items():
  print(f"{name} results:")
  print(df[["host_has_profile_pic","host_identity_verified"]].isna().sum(),"\n")

drop_missing_prices results:
host_has_profile_pic      236
host_identity_verified    236
dtype: int64 

impute_price results:
host_has_profile_pic      257
host_identity_verified    257
dtype: int64 



In [ ]:
# Given the relatively small number of null rows, we have chosen to drop them rather than impute values.

subset_cols = ["host_has_profile_pic", "host_identity_verified"]

for name, df in datasets.items():
    # Drop rows with NA in any of the subset columns
    df.dropna(subset=subset_cols, inplace=True)

    # Show remaining NA counts for those columns
    print(f"{name} results:")
    print(df[subset_cols].isna().sum(),"\n")

drop_missing_prices results:
host_has_profile_pic      0
host_identity_verified    0
dtype: int64 

impute_price results:
host_has_profile_pic      0
host_identity_verified    0
dtype: int64 



## CLEAN UP MISSING "bed" AND "bedroom" ATTRIBUTES

In [ ]:
''' There are three measures for how many people a property fits:
1. "accommodates" = max number of guests
2. "bedrooms" = number of bedrooms
3. "beds" = number of beds '''

for name, df in datasets.items():
  print(f"{name} results:\n",df[['accommodates','bedrooms','beds']].isna().sum(),"\n")

# "accommodates" has no missingness, while the others do, so we use combinations of them to impute values
# We will use all for the training

# THIS RESULT COULD INDICATE A PATTERN OF MISSINGNESS, SINCE ROWS WITH MISSING PRICES ALSO HAVE OTHER MISSING ATTRIBUTES.
# WE COULD CHECK IF THERE IS MISSINGNESS ACROSS, FOR EXAMPLE, A CERTAIN TYPE OF LISTING, THEN THE RESULTS WILL BE SKEWED.

drop_missing_prices results:
 accommodates    0
bedrooms        1
beds            1
dtype: int64 

impute_price results:
 accommodates       0
bedrooms         158
beds            2240
dtype: int64 



In [ ]:
# WE ARE DOING THE FOLLOWING ONLY ON THE PRICE IMPUTATION DF, SINCE THE DROP PRICE DF HAS ONLY 1 NULL VALUE.
# Only use rows where all three variables are valid (non-missing, non-zero)
valid = df_price_imputation[
    (df_price_imputation['accommodates'] > 0) &
    (df_price_imputation['beds'] > 0) &
    (df_price_imputation['bedrooms'] > 0)
]

# Compute ratios
valid['guests_per_bed'] = valid['accommodates'] / valid['beds']
valid['beds_per_bedroom'] = valid['beds'] / valid['bedrooms']
valid['guests_per_bedroom'] = valid['accommodates'] / valid['bedrooms']

# Get averages and medians
summary = valid[['guests_per_bed', 'beds_per_bedroom', 'guests_per_bedroom']].agg(['mean', 'median'])
print("Statistics before imputation:\n", summary)

# Use medians for integers
guests_per_bed = summary.loc['median', 'guests_per_bed']
beds_per_bedroom = summary.loc['median', 'beds_per_bedroom']
guests_per_bedroom = summary.loc['median', 'guests_per_bedroom']

# Impute median values where missing
# For missing beds but nonmissing bedrooms, impute median beds per bedroom
df_price_imputation.loc[df_price_imputation['beds'].isna() & df_price_imputation['bedrooms'].notna(), 'beds'] = df_price_imputation['bedrooms'] * beds_per_bedroom

# For remaining missing beds, divide max guest count by median guests per bed
df_price_imputation.loc[df_price_imputation['beds'].isna(), 'beds'] = df_price_imputation['accommodates'] / guests_per_bed

# For missing bedrooms but nonmissing bedrooms, divide beds by beds per bedroom
df_price_imputation.loc[df_price_imputation['bedrooms'].isna() & df_price_imputation['beds'].notna(), 'bedrooms'] = df_price_imputation['beds'] / beds_per_bedroom

# For remaining missing bedrooms, divide max guest count by median guests per bedroom
df_price_imputation.loc[df_price_imputation['bedrooms'].isna(), 'bedrooms'] = df_price_imputation['accommodates'] / guests_per_bedroom

# Re-check missingness
print("\nRE-CHECKING MISSINGNESS ON PRICE IMPUTATION DATAFRAME:\n", df_price_imputation[['accommodates','bedrooms','beds']].isna().sum())

# DROP THE 1 MISSING VALUE FOR THE DROP PRICE DATAFRAME
df_drop_price = df_drop_price.dropna(subset=["beds"])
df_drop_price = df_drop_price.dropna(subset=["bedrooms"])

# Show the results
print("\nRE-CHECKING MISSINGNESS ON DROP PRICE DATAFRAME:\n", df_drop_price[['accommodates','bedrooms','beds']].isna().sum())



Statistics before imputation:
         guests_per_bed  beds_per_bedroom  guests_per_bedroom
mean          1.983956          1.225566            2.261351
median        2.000000          1.000000            2.000000

RE-CHECKING MISSINGNESS ON PRICE IMPUTATION DATAFRAME:
 accommodates    0
bedrooms        0
beds            0
dtype: int64

RE-CHECKING MISSINGNESS ON DROP PRICE DATAFRAME:
 accommodates    0
bedrooms        0
beds            0
dtype: int64


## CLEAN UP MISSING "bathroom" AND "bathroom_text" ATTRIBUTES

In [ ]:
# Some properties are missing a number of bathrooms in the "bathrooms" attribute. Instead, it's stored as a string in the "bathrooms_text" attribute.

# AGAIN, THIS RESULT COULD INDICATE A PATTERN OF MISSINGNESS, SINCE ROWS WITH MISSING PRICES ALSO HAVE OTHER MISSING ATTRIBUTES.
# WE COULD CHECK IF THERE IS MISSINGNESS ACROSS, FOR EXAMPLE, A CERTAIN TYPE OF LISTING, THEN THE RESULTS WILL BE SKEWED.

for name, df in datasets.items():
  print(f"CHECKING NULL COUNTS ON {name} DATAFRAME:\n", df[["bathrooms", "bathrooms_text"]].isna().sum().to_frame(name="Null Count"))
  print(f"\nCHECKING MISSINGNESS ON {name} DATAFRAME:\n", df.loc[df["bathrooms"].isna(),["bathrooms","bathrooms_text"]].head(10),"\n")

CHECKING NULL COUNTS ON drop_missing_prices DATAFRAME:
                 Null Count
bathrooms                2
bathrooms_text           2

CHECKING MISSINGNESS ON drop_missing_prices DATAFRAME:
        bathrooms bathrooms_text
106          NaN            NaN
19459        NaN            NaN 

CHECKING NULL COUNTS ON impute_price DATAFRAME:
                 Null Count
bathrooms             2241
bathrooms_text           2

CHECKING MISSINGNESS ON impute_price DATAFRAME:
      bathrooms bathrooms_text
7          NaN      1.5 baths
8          NaN  1 shared bath
9          NaN      1.5 baths
38         NaN         1 bath
44         NaN         1 bath
54         NaN         1 bath
61         NaN         1 bath
83         NaN         1 bath
98         NaN         1 bath
106        NaN            NaN 



In [ ]:
# WE ARE DOING THE FOLLOWING ONLY ON THE PRICE IMPUTATION DF, SINCE THE DROP PRICE DF HAS ONLY 2 NULL VALUES.
# Extract the numeric part from bathrooms_text
bathrooms_from_text = (
    df_price_imputation["bathrooms_text"]
    .astype(str)
    .str.extract(r'(\d+(\.\d+)?)')[0]   # capture integers or decimals
    .astype(float)
)

# Fill missing values in bathrooms with extracted numbers
df_price_imputation["bathrooms"] = df_price_imputation["bathrooms"].fillna(bathrooms_from_text)

In [ ]:
df_price_imputation["bathrooms"].isna().sum()

np.int64(21)

In [ ]:
# WE ARE DOING THE FOLLOWING ONLY ON THE PRICE IMPUTATION DF, SINCE THE DROP PRICE DF HAS ONLY 2 NULL VALUES.
# Drop rest of missing values since the row count is so low

df_price_imputation = df_price_imputation.dropna(subset=['bathrooms'])

In [ ]:
df_price_imputation["bathrooms"].isna().sum()

np.int64(0)

In [ ]:
# Show the pre results
print("\nCHECKING NULL COUNTS ON PRICE DROP DATAFRAME BEFORE:\n", df_drop_price[["bathrooms", "bathrooms_text"]].isna().sum().to_frame(name="Null Count"))

# DROP THE 2 MISSING VALUES FOR THE DROP PRICE DATAFRAME
df_drop_price = df_drop_price.dropna(subset=["bathrooms"])
df_drop_price = df_drop_price.dropna(subset=["bathrooms_text"])

# Show the post results
print("\nCHECKING NULL COUNTS ON PRICE DROP DATAFRAME AFTER:\n", df_drop_price[["bathrooms", "bathrooms_text"]].isna().sum().to_frame(name="Null Count"))


CHECKING NULL COUNTS ON PRICE DROP DATAFRAME BEFORE:
                 Null Count
bathrooms                2
bathrooms_text           2

CHECKING NULL COUNTS ON PRICE DROP DATAFRAME AFTER:
                 Null Count
bathrooms                0
bathrooms_text           0


## TRANSFORM "description" ATTRIBUTE BY ADDING "description_missing" AND "description_length" ATTRIBUTES

In [ ]:
# Description has some missing values
for name, df in datasets.items():
  print(f"CHECK MISSING DESCRIPTION VALUE COUNT FOR {name} DATAFRAME:\n", df["description"].isna().sum(),"\n")

CHECK MISSING DESCRIPTION VALUE COUNT FOR drop_missing_prices DATAFRAME:
 134 

CHECK MISSING DESCRIPTION VALUE COUNT FOR impute_price DATAFRAME:
 176 



In [ ]:
for name, df in datasets.items():
  df['description_missing'] = df['description'].isna().astype(int) # Make flag for missing description
  df['description_length'] = df['description'].fillna('').str.len() # Add description length

# Show the results of adding the above columns
for name, df in datasets.items():
  print(f"\ndescription_missing HEAD FOR {name} DATAFRAME:\n", df['description_missing'].head(10))
  print(f"\ndescription_length HEAD FOR {name} DATAFRAME:\n", df['description_length'].head(10))

# We are now dropping the "description" attribute, as we no longer need it (we are not doing any text analysis).
for name, df in datasets.items():
  datasets[name] = df.drop(columns=["description"])


description_missing HEAD FOR drop_missing_prices DATAFRAME:
 2     0
4     0
5     0
10    0
14    0
16    0
17    0
18    0
21    1
22    1
Name: description_missing, dtype: int64

description_length HEAD FOR drop_missing_prices DATAFRAME:
 2     482
4     412
5      75
10     54
14    170
16    290
17    230
18    352
21      0
22      0
Name: description_length, dtype: int64

description_missing HEAD FOR impute_price DATAFRAME:
 2     0
4     0
5     0
7     0
8     0
9     0
10    0
14    0
16    0
17    0
Name: description_missing, dtype: int64

description_length HEAD FOR impute_price DATAFRAME:
 2     482
4     412
5      75
7     558
8     515
9     471
10     54
14    170
16    290
17    230
Name: description_length, dtype: int64


## TRANSFORM "host_since" ATTRIBUTE BY ADDING "host_tenure_days" ATTRIBUTE

In [ ]:
# Data has a host_since feature, so we use it to create a "host_tenure_days" column for each dataframe.

for name, df in datasets.items():
  df['host_since'] = pd.to_datetime(df['host_since'], errors='coerce')
  latest_scrape = pd.to_datetime(df['last_scraped']).max() # Isolate the latest scrape date for calculation
  df['host_tenure_days'] = (latest_scrape - df['host_since']).dt.days # Create new "host_tenure_days" attribute

# Drop "host_since" and "last_scraped" columns
for name, df in datasets.items():
  datasets[name] = df.drop(columns=["host_since", "last_scraped"])

In [ ]:
for name, df in datasets.items():
  print(f"{name} results: ",df["host_tenure_days"].isna().sum())

drop_missing_prices results:  0
impute_price results:  0


## TRANSFORM "host_about" ATTRIBUTE BY ADDING "host_about_missing" AND "host_about_length" ATTRIBUTES

In [ ]:
for name, df in datasets.items():
  df['host_about_missing'] = df['host_about'].isna().astype(int) # Make flag for host about
  df['host_about_length'] = df['host_about'].fillna('').str.len() # Host about length

# We are now dropping the "host_about" attribute, as we are not doing any text analysis
for name, df in datasets.items():
  datasets[name] = df.drop(columns=["host_about"])

## TRANSFORM "host_response_rate" AND "host_acceptance_rate" BY ADDING "...missing" ATTRIBUTES

In [ ]:
# Format as float, add missing flag, fill with median

for name, df in datasets.items():
  for col in ["host_response_rate", "host_acceptance_rate"]:
      temp = df[col].astype(str).str.strip().str.rstrip('%').replace('', np.nan)
      df[col] = pd.to_numeric(temp, errors='coerce')
      df[f"{col}_missing"] = df[col].isna().astype(int)
      df[col] = df[col].fillna(df[col].median())

## TRANSFORM "host_response_time" ATTRIBUTE BY ADDING CATEGORY FOR MISSING VALUES CALLED "unknown"

In [ ]:
# Response time is a category
# Fill unknown or missing response time with category "unknown"

for name, df in datasets.items():
  df['host_response_time'] = df['host_response_time'].fillna('unknown')

## TRANSFORM "neighborhood_overview" ATTRIBUTE BY ADDING "neighborhood_overview_length" AND "neighborhood_overview_missing" ATTRIBUTES

In [ ]:
for name, df in datasets.items():
  df['neighborhood_overview_missing'] = df['neighborhood_overview'].isna().astype(int) # Make flag for neighboorhood overview
  df['neighborhood_overview_length'] = df['neighborhood_overview'].fillna('').str.len() # Neighborhood overview length

# We are now dropping the "neighborhood_overview" attribute, as we are not doing any text analysis
for name, df in datasets.items():
  df.drop(columns=["neighborhood_overview"], inplace=True)

# Print results
for name, df in datasets.items():
  print(f"{name} results: ",df[["neighborhood_overview_missing","neighborhood_overview_length"]].isna().sum())

drop_missing_prices results:  neighborhood_overview_missing    0
neighborhood_overview_length     0
dtype: int64
impute_price results:  neighborhood_overview_missing    0
neighborhood_overview_length     0
dtype: int64


In [ ]:
list(df_drop_price)

['last_scraped',
 'description',
 'neighborhood_overview',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'availability_30',
 'number_of_reviews',
 'instant_bookable',
 'calculated_host_listings_count']

## TRANSFORM "host_location" ATTRIBUTE BY ADDING "host_location_missing" ATTRIBUTE

In [ ]:
for name, df in datasets.items():
  df['host_location_missing'] = df['host_location'].isna().astype(int) # Make flag for host location

# We are now dropping the "host_location" attribute, as we are not doing any text analysis
for name, df in datasets.items():
  df.drop(columns=["host_location"], inplace=True)

# Print results
for name, df in datasets.items():
  print(f"{name} results: ",df[["host_location_missing"]].isna().sum())

KeyError: 'host_location'

In [ ]:
df_drop_price.sort_values(by="price", ascending=True).head(10)
df_drop_price.shape

(7632, 30)

## [TEMP] DROP COLUMNS THAT WE WILL COME BACK TO LATER

In [ ]:
list(df_drop_price)

['last_scraped',
 'description',
 'neighborhood_overview',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'availability_30',
 'number_of_reviews',
 'instant_bookable',
 'calculated_host_listings_count']

## CREATE HEX BINS FROM LONG/LAT TO ADD A MORE GRANULAR LOCATION ATTRIBUTE

In [ ]:
# Neighborhood data is not very granular
# Long and lat used to create hexbins

import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt

# --- 1. Create GeoDataFrame from lat / lon ---

gdf = gpd.GeoDataFrame(
    df.copy(),
    geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
    crs="EPSG:4326"   # WGS84 (lat/lon)
)

# Project to a metric CRS (UTM zone – here: 32N, good for Denmark/southern Sweden)
gdf = gdf.to_crs(epsg=32632)


# --- 2. Helper: build a single regular hexagon around a center ---

def make_hexagon(cx, cy, radius):
    """
    Create a pointy-top regular hexagon centered at (cx, cy)
    with given radius (distance from center to each vertex).
    """
    # Pointy-top: start at 30° and step by 60°
    angles = np.deg2rad(np.arange(0, 360, 60) + 30)
    coords = [(cx + radius * np.cos(a), cy + radius * np.sin(a)) for a in angles]
    return Polygon(coords)


# --- 3. Build a hex grid over the extent of gdf ---

def make_hex_grid(gdf, radius):
    """
    Create a pointy-top hexagon grid covering the extent of gdf.
    radius = distance from hex center to each vertex (in CRS units, e.g. meters).
    """
    xmin, ymin, xmax, ymax = gdf.total_bounds

    # Pointy-top spacing (Red Blob Games):
    # horizontal distance between centers = sqrt(3) * radius
    # vertical distance between rows = 1.5 * radius
    dx = np.sqrt(3) * radius
    dy = 1.5 * radius

    cols = np.arange(xmin - dx, xmax + dx, dx)
    rows = np.arange(ymin - dy, ymax + dy, dy)

    hexes = []
    for row_idx, cy in enumerate(rows):
        for col_idx, cx in enumerate(cols):
            # Offset every second row by half the horizontal spacing
            cx_shifted = cx + (dx / 2.0 if row_idx % 2 == 1 else 0.0)
            hex_poly = make_hexagon(cx_shifted, cy, radius)
            hexes.append(hex_poly)

    hex_grid = gpd.GeoDataFrame(
        {"hex_id": range(len(hexes))},
        geometry=hexes,
        crs=gdf.crs
    )
    return hex_grid


# --- 4. Generate hex grid + optional trimming ---

hex_radius = 250  # meters
hex_grid = make_hex_grid(gdf, hex_radius)

# Optional trimming to a buffered convex hull of listings
study_area = gdf.geometry.union_all().convex_hull.buffer(2 * hex_radius)
hex_grid = hex_grid[hex_grid.intersects(study_area)].reset_index(drop=True)
hex_grid["hex_id"] = hex_grid.index  # reindex after trimming

# --- 5. Spatial join: assign each listing to a hex ---

joined = gpd.sjoin(
    gdf,
    hex_grid[["hex_id", "geometry"]],
    how="left",
    predicate="within"   # use "intersects" if you see edge-cases
)

# Join hex_id to the original df

merged = df.merge(joined[["id", "hex_id"]], on="id")


KeyError: "['id'] not in index"

In [ ]:
# --- Visualization: check that hexes look like hexes ---

fig, ax = plt.subplots(figsize=(8, 8))

# plot hex outlines
hex_grid.boundary.plot(ax=ax, linewidth=0.5)

# plot listing points
gdf.plot(ax=ax, markersize=3, color="red", alpha=0.7)

ax.set_title("Listings and true hex grid")
ax.set_axis_off()
plt.show()

### Mapping hexes

In [ ]:
import folium

# Convert hexes and points to WGS84 (lat/lon)
hex_wgs = hex_grid.to_crs(epsg=4326)
pts_wgs = gdf.to_crs(epsg=4326)

center_lat = pts_wgs.geometry.y.mean()
center_lon = pts_wgs.geometry.x.mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=11)

folium.GeoJson(
    hex_wgs,
    name="Hex grid",
    style_function=lambda feature: {
        "fillColor": "none",
        "color": "blue",
        "weight": 1,
        "fillOpacity": 0.1,
    },
).add_to(m)

sample_pts = pts_wgs.sample(min(2000, len(pts_wgs)), random_state=0)

for _, row in sample_pts.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=2,
        color="red",
        fill=True,
        fill_opacity=0.7,
    ).add_to(m)

m

# PLACEHOLDER FOR MODELING

In [ ]:
# --------------------------------------------------
# 0. Wrap your two datasets in a dictionary - NOTE TO GROUP: VALIDATE THAT THE 2 DATASETS WORK, OR PROCEED WITH JUST 1 AT A TIME.
# --------------------------------------------------
datasets = {
    "price_imputation": df_price_imputation,
    "drop_price": df_drop_price,
}

# Replace this with the actual column name in your data
target_col = "occupancy_30"   # e.g. number of booked days in next 30

# --------------------------------------------------
# 1. Define candidate models (supervised regressors) - NOTE TO GROUP: ANY OTHER MODELS TO CONSIDER?
# --------------------------------------------------
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "KNN": KNeighborsRegressor(),
    "DecisionTree": DecisionTreeRegressor(),
    "RandomForest": RandomForestRegressor(),
    "GradientBoosting": GradientBoostingRegressor(),
}

# --------------------------------------------------
# 2. Define hyperparameter grids for each model - NOTE TO GROUP: GET THE PIPELINE TO CONSIDER ALL HYPERPARAMETERS INSTEAD OF JUST ARBITRARY NUMBERS
# --------------------------------------------------
param_grids = {
    "LinearRegression": {},  # no hyperparameters

    "Ridge": {
        "model__alpha": [0.1, 1.0, 10.0],
    },

    "Lasso": {
        "model__alpha": [0.001, 0.01, 0.1],
    },

    "KNN": {
        "model__n_neighbors": [3, 5, 7, 15],
        "model__weights": ["uniform", "distance"],
    },

    "DecisionTree": {
        "model__max_depth": [3, 5, 10, None],
        "model__min_samples_split": [2, 10, 30],
    },

    "RandomForest": {
        "model__n_estimators": [100, 300],
        "model__max_depth": [5, 10, None],
    },

    "GradientBoosting": {
        "model__learning_rate": [0.01, 0.1],
        "model__n_estimators": [100, 200],
    },
}

# --------------------------------------------------
# 3. Run the full pipeline for each dataset -
# --------------------------------------------------
all_results = []

for ds_name, df in datasets.items():
    print(f"\n================ {ds_name} ================\n")

    # --- 3.1 Split into features (X) and target (y)
    X = df.drop(columns=[target_col])
    y = df[target_col]

    # --- 3.2 Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # --- 3.3 Identify numeric vs categorical columns
    numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = X_train.select_dtypes(
        include=["object", "category"]
    ).columns.tolist()

    # --- 3.4 Preprocessing: scaling + one-hot encoding
    preprocess = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numeric_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ]
    )

    # --- 3.5 Loop over models and run GridSearchCV
    for model_name, model in models.items():
        print(f"Fitting model: {model_name} on {ds_name}...")

        pipe = Pipeline(steps=[
            ("preprocess", preprocess),
            ("model", model),
        ])

        grid = GridSearchCV(
            estimator=pipe,
            param_grid=param_grids[model_name],
            cv=5,
            scoring="neg_mean_absolute_error",
            n_jobs=-1,
        )

        grid.fit(X_train, y_train)

        best_model = grid.best_estimator_

        # --- 3.6 Evaluate on test set
        y_pred = best_model.predict(X_test)

        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred, squared=False)

        # Guard against division by zero in MAPE
        non_zero_mask = y_test != 0
        if non_zero_mask.sum() > 0:
            mape = np.mean(
                np.abs((y_test[non_zero_mask] - y_pred[non_zero_mask]) / y_test[non_zero_mask])
            ) * 100
        else:
            mape = np.nan

        all_results.append({
            "Dataset": ds_name,
            "Model": model_name,
            "Best Params": grid.best_params_,
            "MAE": mae,
            "RMSE": rmse,
            "MAPE": mape,
        })

# --------------------------------------------------
# 4. Collect and inspect results
# --------------------------------------------------
results_df = pd.DataFrame(all_results)
results_df = results_df.sort_values(by=["Dataset", "MAE"])
print(results_df)




## WE NEED TO CONSIDER EVALUATING OUR MODEL ON WHETHER OR NOT IT PERFORMS BETTER/WORSE FOR CERTAIN TYPES LISTINGS (E.G., LESS ACCURATE FOR LISTINGS WITH HIGHER)

## REDO DATA WRANGLING TO USE ONE DATASET UNTIL WE REACH PRICE, THEN DO THE SPLIT

## FIX HEXBIN LOCATION VARIABLE

## FIGURE OUT AMENITIES VARIABLE
